In [45]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
from pandas.io.json import json_normalize
import nltk
import re
from rouge import Rouge 
from collections import Counter
from geotext import GeoText
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
import string
from gensim.summarization.summarizer import summarize

from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer #We're choosing Lexrank, other algorithms are also built in
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

import articleDateExtractor


global str

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package maxen

In [2]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
# nltk.internals.config_java("C:/Program Files/Java/jdk1.8.0_181/bin")

import os
java_path = "C:/Program Files/Java/jdk1.8.0_181/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('C:/Users/anike/Downloads/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz','C:/Users/anike/Downloads/stanford-ner-2018-10-16/stanford-ner-3.9.2.jar', encoding='utf-8')


# Classifier 

In [3]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
import spacy
import re
import csv
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score

C:\Users\anike\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\anike\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
lemma = nltk.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess(rawText):
    cleanText = re.sub(r'[^\w\s]','',rawText.lower())
    tokens = word_tokenize(cleanText)
    tokens = [token for token in tokens if not token in stop_words]
    tokens = [lemma.lemmatize(token) for token in tokens ]
    processedText = (" ".join(tokens))
    return processedText

# Importing dataset

In [41]:
data = open(r'C:\Users\anike\indiaCorpus_01_04.csv')
# data = open(r'C:\Users\anike\indiaCorpus_2010_2019.csv')

target, notes = [], []
reader = csv.reader(data, delimiter=",")
next(reader, None)
for i, line in enumerate(reader):
    target.append(line[1])
    rawText = line[0]
    notes.append(preprocess(rawText))

dataSet = pd.DataFrame()
dataSet['notes'] = notes
dataSet['target'] = target
dataSet = dataSet.sample(frac=1).reset_index(drop=True)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(dataSet['notes'], dataSet['target'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
vector.fit(dataSet['notes'])
train_x =  vector.transform(train_x)
valid_x =  vector.transform(valid_x)

# Naieve Bayes Event Classifier

In [42]:

clf = MultinomialNB().fit(train_x, train_y)
predicted = clf.predict(valid_x)
print(accuracy_score(valid_y,predicted))

0.8591639871382637


# SVM Event Classifier

In [43]:
from sklearn import svm
SVM = svm.LinearSVC()
SVM.fit(train_x, train_y)
predict = SVM.predict(valid_x)
accuracy_score(valid_y,predict)

0.9421221864951769

# Random Forest Event Classifier

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(train_x, train_y)
predict = rf.predict(valid_x)
accuracy_score(valid_y,predict)

0.9131832797427653

# Actor Classifier

In [13]:
data = open(r'C:\Users\anike\indiaActorsCorpus2_2010_2019.csv')

target, notes = [], []
reader = csv.reader(data, delimiter=",")
next(reader, None)
for i, line in enumerate(reader):
    target.append(line[1])
    rawText = line[0]
    notes.append(preprocess(rawText))

df = pd.DataFrame()
df['notes'] = notes
df['target'] = target
df['target'] = df['target'].replace({'CPI (Maoist): Communist Party of India (Maoist)':'CPI'})
df['target'] = df['target'].replace({'BJP: Bharatiya Janata Party':'BJP'})
df['target'] = df['target'].replace({'INC: Indian National Congress':'INC'})
df['target'] = df['target'].replace({'Protesters':'Protesters'})
df['target'] = df['target'].replace({'Rioters':'Rioters'})

df = df.sample(frac=1).reset_index(drop=True)

actrain_x, acvalid_x, actrain_y, acvalid_y = model_selection.train_test_split(df['notes'], df['target'])

acencoder = preprocessing.LabelEncoder()
actrain_y = acencoder.fit_transform(actrain_y)
acvalid_y = acencoder.fit_transform(acvalid_y)

acvector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=3000)
acvector.fit(df['notes'])
actrain_x =  acvector.transform(actrain_x)
acvalid_x =  acvector.transform(acvalid_x)



# Random Forest Actor Classifier

In [14]:

from sklearn.ensemble import RandomForestClassifier
actorRf = RandomForestClassifier(n_estimators=50)
actorRf.fit(actrain_x, actrain_y)
acpredict = actorRf.predict(acvalid_x)
accuracy_score(acvalid_y,acpredict)

0.9460240587408217

# SVM Actor Classifier

In [15]:
from sklearn import svm
acSVM = svm.LinearSVC()
acSVM.fit(actrain_x, actrain_y)
predict = acSVM.predict(acvalid_x)
accuracy_score(acvalid_y,predict)

0.9524293079206374

In [19]:
LIMIT = 4
data = {}
data['newspapers'] = {}
articlesData ={}
collections = {
    'total' : []
}


In [87]:
# with open('newspaperLinks.json') as file:
with open('newspaperLinks.json') as file:
    sources = json.load(file)
sources

{'TOI': {'rss': 'https://timesofindia.indiatimes.com/rssfeeds/-2128936835.cms',
  'link': 'https://timesofindia.indiatimes.com/elections/news'},
 'Hindustan Times': {'rss': 'https://www.hindustantimes.com/rss/india/rssfeed.xml',
  'link': 'https://www.hindustantimes.com/'},
 'India Today': {'rss': 'https://www.indiatoday.in/rss/1206578',
  'link': 'https://indianexpress.com/?s=politics'},
 'FPJ': {'rss': 'https://www.freepressjournal.in/feed',
  'link': 'https://www.news18.com/politics/?ref=topnav'},
 'Business Standard': {'rss': 'https://www.business-standard.com/rss/latest.rss',
  'link': 'https://www.business-standard.com/lok-sabha-elections-2019'}}

In [74]:
def summarizeText(text):
    string=text
    parser = PlaintextParser.from_string(string, Tokenizer("english"))

    summarizer = LexRankSummarizer()
    summary_lex = summarizer(parser.document, 1) #Summarize the document with 5 sentences
    summary_1=''
    for line in summary_lex:
        #summary_1+=line
        #print(line)
        summary_1+=str(line)
#     print(summary_1)

#     lsa_summarizer = LsaSummarizer()
#     summary_lsa =lsa_summarizer(parser.document,3)
#     summary_2=''
#     print("\n")
#     for line in summary_lsa:
#         summary_2+=str(line)
#     print(summary_2)

#     luhn_summarizer = LuhnSummarizer() 
#     luhn_summarizer.stop_words = ("I",  "me", "is", "am", "the","than", "that", "this","an","you", "are")
#     summary_3=''
#     print("\n")
#     for line in luhn_summarizer(parser.document, 3):
#         summary_3+=str(line)
#     print(summary_3)
    return summary_1

def getLocation(ents, text):
    loc = ""
#     locList = []
    items = [x.text for x in ents if x.label_ in 'GPE' or x.label_ in 'LOC']
    loc_ = [Counter(items).most_common(1)]
    places = GeoText(text)
    topPlace = [Counter(places.cities).most_common(1)]
    
    places = (places.cities)
#     text = re.sub(r'[^\w\s]','',text)
#     tokenized_text = word_tokenize(text)
#     classified_text = st.tag(tokenized_text)
#     stanfordLocations =[]
#     for x in classified_text:
#         if(x[1]=="LOCATION"):
#             stanfordLocations.append(x[0])
#     topPlace = [Counter(stanfordLocations).most_common(1)]
#     locList = [i for i in stanfordLocations if i in items]
    locList = [i for i in places if i in items]
    total = items+locList 
    location_ = [Counter(total).most_common(1)]
    if len(location_) == 0:
        if(len(loc_))!=0:
            loc = loc_[0][0][0]
        elif (len(topPlace)!=0):
            loc = topPlace[0]
        else:
            loc = ""
    else:
        if(len(location_))!=0:
            if(len(location_[0]))!=0:
                if(len(location_[0][0]))!=0:
                    loc = location_[0][0][0]
        else:
            loc = "unknown"
    return loc

def getEventType(tokens):
#     eventTypes ={
#     'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood'],
#     'riotsProtest':['protest','riot']
#     }
    eventType = "other"
    for event, types in eventTypes.items():
        tag = [token for token in tokens if any(typ in token for typ in types)]
        if len(tag) != 0:
            eventType = event
            break
    return eventType

# News Scraper and Classifier

In [88]:
collections = {
    'total' : []
}
from time import mktime
from datetime import datetime
from lexrank import STOPWORDS, LexRank

event_type = ""
data = {}
data['newspapers'] = {}
articlesData ={}
collectedNews = {}
events = ['arrest','arrested','violence against civilians','rioting','protesting','riots','protest','battles','violence','remote violence','assault','attack','bloodshed','brutality','clash','confusion','cruelty','disorder','disturbance','fighting','rampage','struggle','terrorism','abandon','acuteness','bestiality','blowup','coercion','compulsion','constraint','destructiveness','duress','ferocity','fervor','fierceness','flap','frenzy','fury','fuss','harshness','murderousness','onslaught','passion','power','roughness','ruckus','rumble','savagery','scam','severity','sharpness','storm','storminess','tumult','turbulence','uproar','vehemence','wildness','brute','force','foul','play','raging','anarchy','brawl','disturbance','lawlessness','protest','storm','strife','trouble','turbulence','turmoil','uproar','anarchism','burst','commotion','confusion','distemper','flap','fray','free-for-all','fuss','hassle','misrule','mix-up','quarrel','racket','row','ruckus','ruction','rumble','rumpus','run-in','scene','shivaree','shower','snarl','stir','to-do','tumult','brannigan','mob','violence','street','fighting','wingding','action','assault','attack','bloodshed','bombing','campaign','clash','combat','conflict','crusade','encounter','fighting','hostility','skirmish','strife','struggle','war','warfare','barrage','brush','carnage','contention','engagement','fray','havoc','onset','onslaught','press','ravage','scrimmage','sortie','blitzkreig']
politics= ["politicize", "political affairs", "administration", "gandhi", "Modi", "politics", "politicians", "elections", "votes", "voters", "BJP", "Congress", "Supreme Court","Nationalism","polls","poll","voting"]
violence = ["violence", "criminal violence", "criminal", "crime", "criminal party", "election violence", "electoral violence","mob killing", "vigilante", "riots", "killing" ]
parties = ['bharatiya janata party', 'bjp', 'b.j.p.', 'bhaujan samaj party', 'bsp', 'b.s.p.', 'indian national congress', 'inc', 'i.n.c.', 'congress', 'nationalist congress party', 'ncp', 'n.c.p.','communist party of india','cpi','c.p.i.','aam aadmi party','aap','a.a.p.','samajwadi party','shiv sena','shs','rashtriya janta dal','rjd','r.j.d.']
    
eventTypes ={
'riots':['riot'],
'protests':['protest','march'], 
'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood']
}



count = 1

for source, link in sources.items():
    print(source)
    count = 1
    if 'rss' in link:
        parsedFeed = fp.parse(link['rss'])
        collected = {
            'rss': link['rss'],
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.entries:
            
            try:

                articlesData ={}
                articles = Article(feed['links'][0]['href'])
                articles.download()
                articles.parse()
                articles.nlp()
                strg = re.sub(r'[^\w\s]','',articles.text)
                articleTokens = nlp(strg)

                tokens =[]
                for token in articleTokens:
                    tokens.append(token.text.lower())

                txt = str(articles.text).split()
                flag = True
                for i in txt:
                    for j in events:
                        if i==j:
                            event_type = j
                            flag = False
                            break
                        else:
                            event_type = "other"
                    if not flag:
                        break
                keyWords = articles.keywords

                eventType = "other"
                if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                    print(articles.title)
                    articlesData['title'] = articles.title

                    article = nlp(articles.text)
                    ###Summary
                    sentences = [x for x in article.sents]
                    sentences = list( map(str, sentences) )
                    lxr = LexRank(articles.text)
                    lxr.get_summary(sentences)
                    summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                    summary = (" ".join(summary))
                    
                    rawSummary = [preprocess(summary)]
                    textVector = vector.transform(rawSummary)
                    prediction = rf.predict(textVector)
                    print(encoder.inverse_transform(prediction))
                    articlesData['event'] = encoder.inverse_transform(prediction[0])

                    dates = articleDateExtractor.extractArticlePublishedDate(feed['links'][0]['href'])
                    localFormat = dates.strftime("%m/%d/%Y")
                    summaryDate = dates.strftime("On %d %b,")
                    articlesData['eventDate'] = localFormat
                    articlesData['location'] = getLocation(articleTokens.ents, articles.text)
                    orgs = set(keyWords).intersection(set(parties))

                    if len(orgs) == 0:
                        if(articlesData['event']=="Riots"):
                            taggedOrg = "Rioters"
                        elif(articlesData['event']=="Protests"):
                            taggedOrg = "Protestors"
                        else:
                            taggedOrg = "other"
                    else:
                        taggedOrg=""
                        for org in orgs:
                            taggedOrg+=org+", "
                        taggedOrg=taggedOrg[:-2]

                    articlesData['partiesInvolved'] = taggedOrg
                    articlesData['source'] = source
                    totalSummary = []
                    totalSummary.append(summaryDate)
                    totalSummary.append(summary)
                    extractiveSummary  = (" ".join(totalSummary))
                    articlesData['summary'] = extractiveSummary
                    articlesData['link'] = (feed['links'][0]['href'])
                    collected['article'].append(articlesData)
#                 else:
#                     break
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)
    else:
        parsedFeed = newspaper.build(link['link'], memoize_articles = False)
        collected = {
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.articles:
#             if count > 1000:
#                 break
            try:
                feed.download()
                feed.parse()
                feed.nlp()
                if feed.publish_date is not None:
                    articlesData ={}
                    
                    strg = re.sub(r'[^\w\s]','',feed.text)
                    articleTokens = nlp(strg)
                    tokens =[]
                    for token in articleTokens:
                        tokens.append(token.text.lower())

                        txt = str(feed.text).split()
                        for i in txt:
                            for j in events:
                                if i==j:
                                    event_type = j
                                    break
                                else:
                                    event_type = "other"
                    keyWords = feed.keywords
                    if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                        
                        articlesData['title'] = feed.title
                        article = nlp(feed.text)
                        ###Summary
                        sentences = [x for x in article.sents]
                        sentences = list( map(str, sentences) )
                        lxr = LexRank(feed.text)
                        lxr.get_summary(sentences)
                        summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                        summary = (" ".join(summary))

                        rawSummary = [preprocess(summary)]
                        textVector = vector.transform(rawSummary)
                        prediction = rf.predict(textVector)
                        print(encoder.inverse_transform(prediction))
                        articlesData['event'] = encoder.inverse_transform(prediction[0])
                        
#                         articlesData['event'] = getEventType(tokens)
                        articlesData['eventDate'] = feed.publish_date.strftime('%m/%d/%Y')
                        articlesData['location'] = getLocation(articleTokens.ents,feed.text)
                        
                        orgs = set(keyWords).intersection(set(parties))
                        if len(orgs) == 0:
                            if(articlesData['event']=="Riots"):
                                taggedOrg = "Rioters"
                            elif(articlesData['event']=="Protests"):
                                taggedOrg = "Protestors"
                            else:
                                taggedOrg = "other"
                        else:
                            taggedOrg=""
                            for org in orgs:
                                taggedOrg+=org+", "
                            taggedOrg=taggedOrg[:-2]

                        articlesData['partiesInvolved'] = taggedOrg
                        articlesData['source'] = source
                        totalSummary = []
                        totalSummary.append(feed.publish_date.strftime("On %d %b,"))
                        totalSummary.append(summary)
                        extractiveSummary  = (" ".join(totalSummary))
                        articlesData['summary'] = extractiveSummary
                        articlesData['link'] = link
                        collected['article'].append(articlesData)
#                     else:
#                         break
#                         count +=1
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)

TOI
EC issues notice to Congress leader and Punjab minister Navjot Singh Sidhu for remarks against PM Narendra Modi


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['Protests']


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Extracting date from https://timesofindia.indiatimes.com/india/ec-issues-notice-to-congress-leader-and-punjab-minister-navjot-singh-sidhu-for-remarks-against-pm-narendra-modi/articleshow/69274346.cms
Discoms check theft, raise billing under UDAY but still losses mount in poll season


...

Extracting date from https://www.business-standard.com/article/pti-stories/anti-sikh-remarks-pitroda-accuses-bjp-of-distorting-truth-119051000703_1.html
Ex-Navy officers have exposed Modi's lie about Rajiv Gandhi: Chidambaram


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['Protests']


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Extracting date from https://www.business-standard.com/article/elections/ex-navy-officers-have-exposed-modi-s-lie-about-rajiv-gandhi-chidambaram-119051000771_1.html


In [89]:
collections["total"]

[{'rss': 'https://timesofindia.indiatimes.com/rssfeeds/-2128936835.cms',
  'link': 'https://timesofindia.indiatimes.com/elections/news',
  'article': [{'title': 'EC issues notice to Congress leader and Punjab minister Navjot Singh Sidhu for remarks against PM Narendra Modi',
    'event': 'Protests',
    'eventDate': '05/10/2019',
    'location': 'Sidhu',
    'partiesInvolved': 'Protestors',
    'source': 'TOI',
    'summary': 'On 10 May, The Election Commission on Friday issued a notice to Congress leader and Punjab minister Navjot Singh Sidhu',
    'link': 'https://timesofindia.indiatimes.com/india/ec-issues-notice-to-congress-leader-and-punjab-minister-navjot-singh-sidhu-for-remarks-against-pm-narendra-modi/articleshow/69274346.cms'},
   {'title': 'Discoms check theft, raise billing under UDAY but still losses mount in poll season',
    'event': 'Protests',
    'eventDate': '05/10/2019',
    'location': 'Rs',
    'partiesInvolved': 'Protestors',
    'source': 'TOI',
    'summary': 'O

In [90]:
frame = json_normalize(collections['total'])
rows = frame.shape[0]
frame

,article,link,rss
0,[{'title': 'EC issues notice to Congress leade...,https://timesofindia.indiatimes.com/elections/...,https://timesofindia.indiatimes.com/rssfeeds/-...
1,[],https://www.hindustantimes.com/,https://www.hindustantimes.com/rss/india/rssfe...
2,[{'title': 'Supreme Court reserves order in Ra...,https://indianexpress.com/?s=politics,https://www.indiatoday.in/rss/1206578
3,[],https://www.news18.com/politics/?ref=topnav,https://www.freepressjournal.in/feed
4,[{'title': 'Lok Sabha polls: Army seniors are ...,https://www.business-standard.com/lok-sabha-el...,https://www.business-standard.com/rss/latest.rss


In [91]:
totalDF = pd.DataFrame()
for row in range(0,rows):
    tempDF = pd.DataFrame()
    tempDF = json_normalize(collections['total'][row]['article'])
    totalDF = totalDF.append(tempDF, ignore_index = True) 


# Dataframe that contains scraped news which has been classified and summarized

In [92]:
totalDF

,event,eventDate,link,location,partiesInvolved,source,summary,title
0,Protests,05/10/2019,https://timesofindia.indiatimes.com/india/ec-i...,Sidhu,Protestors,TOI,"On 10 May, The Election Commission on Friday i...",EC issues notice to Congress leader and Punjab...
1,Protests,05/10/2019,https://timesofindia.indiatimes.com/india/disc...,Rs,Protestors,TOI,"On 10 May, The presentation shows the annual r...","Discoms check theft, raise billing under UDAY ..."
2,Protests,05/10/2019,https://timesofindia.indiatimes.com/india/modi...,Kumar,Protestors,TOI,"On 10 May, Kumar rued the lack of trust when i...",Modi seeking to change 'soft & crony'state: Ni...
3,Protests,05/10/2019,https://timesofindia.indiatimes.com/india/rese...,Union,Protestors,TOI,"On 10 May, Otherwise, our past will haunt the ...",Reservations for SCs and the STs is not agains...
4,Protests,05/10/2019,https://timesofindia.indiatimes.com/india/sc-t...,India,Protestors,TOI,"On 10 May, The petitioner has also sought quas...",SC to hear on May 13 plea to advance poll timi...
5,Protests,05/10/2019,https://timesofindia.indiatimes.com/india/will...,saidWe,Protestors,TOI,"On 10 May, Tamil Nadu deputy chief minister O ...",Will urge Guv to release all seven convicts in...
6,Protests,05/10/2019,https://www.indiatoday.in/india/story/supreme-...,India,Protestors,India Today,"On 10 May, Senior advocate Mukul Rohatgi, appe...","Supreme Court reserves order in Rafale review,..."
7,Protests,05/09/2019,https://www.indiatoday.in/india/story/bjp-dema...,Jaipur,bjp,India Today,"On 09 May, After they could not get the money ...",BJP demands CBI probe into Alwar gangrape case
8,Protests,05/09/2019,https://www.indiatoday.in/india/story/alwar-ga...,Mathura,bjp,India Today,"On 09 May, Hours after fourth accused Mahesh w...","Alwar gangrape case: All six accused arrested,..."
9,Protests,05/09/2019,https://www.indiatoday.in/india/story/bjp-resp...,Bhopal,"bjp, congress",India Today,"On 09 May, While mentioning Rajiv Gandhi and h...","BJP respects Rajiv Gandhi, but has right to qu..."


In [93]:
import datetime
now = datetime.datetime.now()
totalDF.to_excel("airData/india/testing_"+now.strftime("%m%d%Y_%H%M%S")+".xlsx")